# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's ThinkPad with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [1]:
# Seed the random number generator so that it will
# produce the same set of random numbers. 
#
# You don't generally need to do this, but it ensures
# that you'll see exactly the same results we do for
# this demo. 
srand(42);

In [2]:
a = rand(10^7) # array of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.533183 
 0.454029 
 0.0176868
 0.172933 
 0.958926 
 0.973566 
 0.30387  
 0.176909 
 0.956916 
 0.584284 
 0.937466 
 0.160006 
 0.422956 
 ⋮        
 0.328823 
 0.116738 
 0.30473  
 0.749187 
 0.301049 
 0.530005 
 0.230942 
 0.563132 
 0.416549 
 0.526003 
 0.851597 
 0.291042 

In [3]:
sum(a) # one expects this is 10^7 * .5 , since the mean of each entry is .5

5.000811321070729e6

# Benchmarking a few implementations in different languages

In [4]:
using BenchmarkTools: @btime, @benchmark

# BenchmarkTools provides the @benchmark and @btime macros,
# which run a given function multiple times to provide a 
# robust estimate of its true runtime. 

INFO: Recompiling stale cache file /home/rdeits/.julia/lib/v0.6/BenchmarkTools.ji for module BenchmarkTools.


#  1. The C language: (8.0 msecs)

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

You can put C code in a Julia session, compile it, and run it.

In [5]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    size_t i;
    for (i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function implemnted above:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [6]:
c_sum(a)

5.000811321070464e6

In [7]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

We can now benchmark the C code directly from Julia:

In [8]:
@btime c_sum($a)

  8.017 ms (0 allocations: 0 bytes)


5.000811321070464e6

# 2. Python's built in `sum` (74 msecs)

In [9]:
# Julia interface to Python:
using PyCall

In [10]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [11]:
pysum(a)

5.000811321070464e6

In [12]:
pysum(a) ≈ sum(a)

true

In [13]:
@btime $pysum($apy_list)

  74.272 ms (17 allocations: 512 bytes)


5.000811321070464e6

# 3. Python: `numpy` (4.5 msec)  

## Takes advantage of hardware "SIMD"

`numpy` is an optimized C library, callable from Python

If it is not installed, install it from Julia as follows:

In [14]:
# using Conda 
# Conda.add("numpy")

In [15]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

numpy_sum(apy_list)

5.0008113210707335e6

In [16]:
numpy_sum(apy_list) ≈ sum(a)

true

In [17]:
@btime $numpy_sum($apy_numpy)

  4.579 ms (17 allocations: 512 bytes)


5.0008113210707335e6

# 4. Python, hand written (319 msec!)

In [18]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
mysum_py = py"""mysum"""o

PyObject <function mysum at 0x7ff03bcf47d0>

In [19]:
mysum_py(apy_list)

5.000811321070464e6

In [20]:
mysum_py(apy_list) ≈ sum(a)

true

In [21]:
@btime $mysum_py($apy_list)

  319.106 ms (17 allocations: 512 bytes)


5.000811321070464e6

# 5. Julia (built-in) (4.4 msec) 

## Written directly in Julia, not in C! 

(and just as fast as numpy's optimized `sum()`)

In [22]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [23]:
@btime sum($a)

  4.401 ms (0 allocations: 0 bytes)


5.000811321070729e6

Note that, like NumPy, Julia's built-in `sum` uses [pairwise summation](https://en.wikipedia.org/wiki/Pairwise_summation) for improved numerical accuracy.

# 6. Julia (hand-written) (8.0 msec, same as hand-written C)

In [24]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [25]:
@btime mysum($a)

  8.018 ms (0 allocations: 0 bytes)


5.000811321070464e6

# 7. Julia (hand-written and using SIMD instructions) (4.2 msec)

In [26]:
function mysum_simd(A)   
    s = 0.0  # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [27]:
@btime mysum_simd($a)

  4.253 ms (0 allocations: 0 bytes)


5.000811321070714e6

## Conclusion

It's often pretty straightforward to write Julia code which is as fast as well-written C code! Loops in Julia are fast, so feel free to use them whenever they help organize or clarify your code. For the innermost loops of performance-critical code, you can exploit `simd` instructions to get even better performance. 